In [1]:
import pandas as pd
import numpy as np
import random
import time
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score

from sklearn.preprocessing import OneHotEncoder


In [2]:
tripdata = pd.read_csv("../../dataset/final_bike_sharing.csv")

In [3]:
selected_features = ['tripduration','start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

In [4]:
df = tripdata[selected_features]

In [5]:
def split_dataset(df):
    X = df.drop('tripduration', axis=1)
    y = df['tripduration']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Train', X_train.shape, y_train.shape)
    print('Test', X_test.shape, y_test.shape)
    return X_train, X_test, y_train, y_test

In [6]:
def getSampleSize(df, perc=1):
    return round(perc/100 * df.shape[0])

def getAnIndex(index):
    li = []
    for i in index:
        li.append(i)
    return li  
        
random.seed(100)
def induceMissingValues(X_train,col_name, perc):
    random.seed(100)
    index = getAnIndex(X_train.index)
    sample_size = getSampleSize(X_train, perc)
    selected_index = random.sample(index, sample_size)
    for i in selected_index:
        if col_name in X_train.columns:
            X_train.at[i,col_name]= np.NaN
        else:
            X_train.loc[i] = np.NaN
    return X_train


In [7]:
def one_hot_encoding(X_train, X_test):
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = X_train.select_dtypes(include=['int', 'float']).columns.tolist()
    ohe = OneHotEncoder(drop='first', sparse=False)
    X_train_encoded = ohe.fit_transform(X_train[categorical_features])
    X_test_encoded = ohe.fit_transform(X_test[categorical_features])
    X_train_new = np.hstack((X_train[numerical_cols].values, X_train_encoded))
    X_test_new = np.hstack((X_test[numerical_cols].values, X_test_encoded))
    return X_train_new, X_test_new

In [8]:
def modelEvaluation(X_train,y_train):
    model_lr = LinearRegression(fit_intercept=True).fit(X_train, y_train) 
    return model_lr

In [9]:
fill_dict ={'start_lat': 40.79865054, 'start_lon': -74.2404433, 'end_lat': 40.79165072 , 'end_lon':-74.28593088,'usertype':'agent',
            'hour': 24, 'temp': 38.6, 'feelslike':49.4,'dew':-25.7,'snowdepth':5.10,'winddir':380.0,'sealevelpressure':819.3,
            'visibility':7.2,'solarradiation': 1109,'conditions': 'muddy','dist': 5.345,'birthyear': 2003,
            'holiday':'unknown','day': 'special','month': 'unknown','gender': 'nonbinary'
        }

In [10]:
def ampute_each_variables(variable, method, X_train, y_train, X_test, y_test):
    X_train  = induceMissingValues(X_train,variable,perc=5)
    print('Train set after Amputation', X_train.shape, y_train.shape)
    print("X_train missing value count of varaible ", X_train.isnull().sum()[variable])
    X_test  = induceMissingValues(X_test,variable,perc=5)
    print('Test set after Amputation', X_test.shape, y_test.shape)
    print("X_test missing value count of varaible ", X_test.isnull().sum()[variable])
    start = datetime.datetime.now()
    if method == "bad":
        print("Simulate Imputation for NA")
        X_train.fillna(fill_dict[variable], inplace=True)
        print('Train  after imputation', X_train.shape, y_train.shape)
        X_test.fillna(fill_dict[variable], inplace=True)
        print('Test  after imputation', X_test.shape, y_test.shape)
    end = datetime.datetime.now()
    comp_time = (end-start)
    print(f"Time taken simulate for variable {var} = ", comp_time)
    list_comp_time.append(comp_time)

    return X_train , y_train, X_test , y_test

In [11]:
variables = ['start_lat','start_lon','end_lat','end_lon','usertype','hour',
                     'temp','feelslike','dew','snowdepth','winddir','sealevelpressure','visibility','solarradiation',
                     'conditions','dist','birthyear','holiday','day','month','gender']

In [12]:
result = {}
list_comp_time =[]
for var in variables:
    print("Now running model is ....", var)
    X_train, X_test, y_train, y_test = split_dataset(df)
    X_train , y_train, X_test , y_test = ampute_each_variables(var,"bad",X_train,y_train,X_test,y_test)
    X_train, X_test = one_hot_encoding(X_train,X_test)
    model_reg = modelEvaluation(X_train,y_train)
    y_pred = model_reg.predict(X_test)
    result[var] = {"MAE":mean_absolute_error(y_test, y_pred),"MSE":mean_squared_error(y_test, y_pred),
                          "RMSE":np.sqrt(mean_squared_error(y_test, y_pred)),"R2":r2_score(y_test, y_pred)}
    del X_train
    del X_test
    del y_train
    del y_test

Now running model is .... start_lat
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train missing value count of varaible  12617
Test set after Amputation (63087, 21) (63087,)
X_test missing value count of varaible  3154
Simulate Imputation for NA
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
Time taken simulate for variable start_lat =  0:00:00.269705
Now running model is .... start_lon
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train missing value count of varaible  12617
Test set after Amputation (63087, 21) (63087,)
X_test missing value count of varaible  3154
Simulate Imputation for NA
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
Time taken simulate for variable start_lon =  0:00:00.269155
Now running model is .... end_lat
Train (252346, 21) (252346,)
Test (63087, 21) (6

Now running model is .... day
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train missing value count of varaible  12617
Test set after Amputation (63087, 21) (63087,)
X_test missing value count of varaible  3154
Simulate Imputation for NA
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
Time taken simulate for variable day =  0:00:00.263975
Now running model is .... month
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set after Amputation (252346, 21) (252346,)
X_train missing value count of varaible  12617
Test set after Amputation (63087, 21) (63087,)
X_test missing value count of varaible  3154
Simulate Imputation for NA
Train  after imputation (252346, 21) (252346,)
Test  after imputation (63087, 21) (63087,)
Time taken simulate for variable month =  0:00:00.232084
Now running model is .... gender
Train (252346, 21) (252346,)
Test (63087, 21) (63087,)
Train set afte

In [15]:
print("averge time to bad imputation: ", 0.291088381)
list_comp_time

averge time to bad imputation:  0.291088381


[datetime.timedelta(microseconds=269705),
 datetime.timedelta(microseconds=269155),
 datetime.timedelta(microseconds=270846),
 datetime.timedelta(microseconds=267328),
 datetime.timedelta(microseconds=292928),
 datetime.timedelta(microseconds=300122),
 datetime.timedelta(microseconds=299870),
 datetime.timedelta(microseconds=365644),
 datetime.timedelta(microseconds=221778),
 datetime.timedelta(microseconds=252300),
 datetime.timedelta(microseconds=319508),
 datetime.timedelta(microseconds=287176),
 datetime.timedelta(microseconds=275819),
 datetime.timedelta(microseconds=338293),
 datetime.timedelta(microseconds=269399),
 datetime.timedelta(microseconds=334090),
 datetime.timedelta(microseconds=392134),
 datetime.timedelta(microseconds=300302),
 datetime.timedelta(microseconds=263975),
 datetime.timedelta(microseconds=232084),
 datetime.timedelta(microseconds=290400)]

In [14]:
result

{'start_lat': {'MAE': 1.571975426531936,
  'MSE': 5.000433629748616,
  'RMSE': 2.2361649379570854,
  'R2': 0.5271581337599838},
 'start_lon': {'MAE': 1.5742714156538424,
  'MSE': 5.010862188888121,
  'RMSE': 2.2384955190681355,
  'R2': 0.5261720074095844},
 'end_lat': {'MAE': 1.5721463768591057,
  'MSE': 5.0021416587966545,
  'RMSE': 2.2365468156952706,
  'R2': 0.5269966222386904},
 'end_lon': {'MAE': 1.5749138576444124,
  'MSE': 5.008412614868598,
  'RMSE': 2.237948304780206,
  'R2': 0.5264036395512438},
 'usertype': {'MAE': 1.5737569179599973,
  'MSE': 5.010686428482828,
  'RMSE': 2.238456260122772,
  'R2': 0.5261886273438149},
 'hour': {'MAE': 1.5730235240278743,
  'MSE': 5.004701530024008,
  'RMSE': 2.237119024554574,
  'R2': 0.5267545603740404},
 'temp': {'MAE': 1.5725792138628913,
  'MSE': 5.000962335500978,
  'RMSE': 2.236283151906524,
  'R2': 0.5271081392528771},
 'feelslike': {'MAE': 1.5724670052286522,
  'MSE': 5.0003295005953,
  'RMSE': 2.236141654858945,
  'R2': 0.527167980